In [ ]:
import cv2
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# ---------------------- Hand Tracking ---------------------- #
def handtrack():
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
    mp_draw = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(0)

    def is_open_hand(landmarks):
        tips_ids = [4, 8, 12, 16, 20]  # Thumb, Index, Middle, Ring, Pinky tips
        open_count = 0
        for tip in tips_ids[1:]:  # Skipping thumb for now
            if landmarks[tip].y < landmarks[tip - 2].y:  # Check if finger is extended
                open_count += 1
        return open_count >= 4

    def is_punch(landmarks):
        tips_ids = [8, 12, 16, 20]
        return all(abs(landmarks[tip].y - landmarks[0].y) < 0.05 for tip in tips_ids)

    while True:
        success, img = cap.read()
        if not success:
            break

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                lm_list = hand_landmarks.landmark

                if is_open_hand(lm_list):
                    cv2.putText(img, "Open Hand 🖐️", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif is_punch(lm_list):
                    cv2.putText(img, "Punch ✊", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow("Hand Tracking + Gesture", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
# ---------------------- Image Recognition ---------------------- #
def imgrecognition():
    model = MobileNetV2(weights='imagenet')

    img_path = 'myimage.png'  # Replace with your image filename
    if not os.path.exists(img_path):
        print(f"[ERROR] Image file '{img_path}' not found.")
        return

    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    predictions = model.predict(img_array)
    decoded = decode_predictions(predictions, top=3)[0]

    print("Top Predictions:")
    for i, (imagenetID, label, prob) in enumerate(decoded):
        print(f"{i+1}. {label}: {prob*100:.2f}%")


# ---------------------- Main ---------------------- #
if __name__ == "__main__":
    print("Choose a mode:")
    print("1 - Hand Tracking (Webcam)")
    print("2 - Image Recognition (Static Image)")

    choice = input("Enter 1 or 2: ").strip()

    if choice == '1':
        handtrack()
    elif choice == '2':
        imgrecognition()
    else:
        print("Invalid input. Please enter 1 or 2.")
        
                   

Choose a mode:
1 - Hand Tracking (Webcam)
2 - Image Recognition (Static Image)


Enter 1 or 2:  1
